In [1]:
# -- driver_run_forecast_LV1.py  --
# master python script to do a full LV1 forecast simulation

import sys
#import pickle
#import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime, timezone
import gc
import resource
#import xarray as xr
#import netCDF4 as nc

##############

sys.path.append('../sdpm_py_util')

import atm_functions as atmfuns
import ocn_functions as ocnfuns
import grid_functions as grdfuns
import util_functions as utlfuns 
import plotting_functions as pltfuns
from util_functions import s_coordinate_4
from get_PFM_info import get_PFM_info
from make_LV1_dotin_and_SLURM import make_LV1_dotin_and_SLURM
from run_slurm_LV1 import run_slurm_LV1


PFM=get_PFM_info()

run_type = 'forecast'
# we will use hycom for IC and BC
ocn_mod = PFM['ocn_model']
print('ocean boundary and initial conditions will be from:')
print(ocn_mod)
# we will use nam_nest for the atm forcing
atm_mod = PFM['atm_model']
print('atm forcing will be from:')
print(atm_mod)
# we will use opendap, and netcdf to grab ocn, and atm data
get_method = 'open_dap_nc'
# figure out what the time is local and UTC
start_time = datetime.now()
utc_time = datetime.now(timezone.utc)
year_utc = utc_time.year
month_utc = utc_time.month
day_utc = utc_time.day
hour_utc = utc_time.hour

print("Starting: driver_run_forecast_LV1: Current local Time =", start_time, "UTC = ",utc_time)

if hour_utc < 12:
    hour_utc=12
    day_utc=day_utc-1  # this only works if day_utc \neq 1

yyyymmdd = "%d%02d%02d" % (year_utc, month_utc, day_utc)
    
#yyyymmdd = '20240717'
# the hour in Z of the forecast, hycom has forecasts once per day starting at 1200Z
hhmm='1200'
forecastZdatestr = yyyymmdd+hhmm+'Z'   # this could be used for model output to indicate when model was initialized.

yyyymmdd = '20240730'
print("Preparing forecast starting on ",yyyymmdd)


# get the ROMS grid as a dict
RMG = grdfuns.roms_grid_to_dict(PFM['lv1_grid_file'])


/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_functions.py:20: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


get_PFM_info(): running onswell
ocean boundary and initial conditions will be from:
hycom
atm forcing will be from:
nam_nest
Starting: driver_run_forecast_LV1: Current local Time = 2024-08-02 15:03:59.591179 UTC =  2024-08-02 22:03:59.591200+00:00
Preparing forecast starting on  20240730


In [3]:
print('before getting OCN, using:')
print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
print('kilobytes')

use_ncks = 1 # flag to get data using ncks. if =0, then a pre saved pickle file is loaded.
if use_ncks == 1:
    OCN = ocnfuns.get_ocn_data_as_dict(yyyymmdd,run_type,ocn_mod,'ncks_para',PFM)
    print('driver_run_forecast_LV1: done with get_ocn_data_as_dict: Current time ',datetime.now() )
    print('after getting OCN with ncks_para, using:')
    print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
    print('kilobytes')
else:
    save_ocn = 0 # if 0, this loads the pickle file. if 1, it saves the pickle file
    import pickle
    # save the OCN dict so that we can restart the python session
    # and not have to worry about opendap timing out
    fnout='/scratch/PFM_Simulations/LV1_Forecast/Forc/ocn_dict_file_2024-07-29T12:00.pkl'
    if save_ocn == 1:
        with open(fnout,'wb') as fp:
            pickle.dump(OCN,fp)
            print('OCN dict saved with pickle')
    else:
        with open(fnout,'rb') as fp:
            OCN = pickle.load(fp)
            print('OCN dict loaded with pickle')
            print('after getting OCN from file, using:')
            print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
            print('kilobytes')


    

before getting OCN, using:
492764
kilobytes
in the parallel ncks switch
Time to get full file using parallel ncks = 89.70 sec
Return code = 0 (0=success, 1=skipped ncks)


KeyboardInterrupt: 

In [3]:
print('before gc.collect and getting OCN_R, using:')
print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
print('kilobytes')
gc.collect()
print('after gc.collect and before OCN_R, using:')
print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
print('kilobytes')
# put the ocn data on the roms grid
print('starting: ocnfuns.hycom_to_roms_latlon(OCN,RMG)')
OCN_R  = ocnfuns.hycom_to_roms_latlon(OCN,RMG)
print('driver_run_forecast_LV1: done with hycom_to_roms_latlon')
# add OCN + OCN_R plotting function here !!!

del OCN

before gc.collect and getting OCN_R, using:
1809016
kilobytes
after gc.collect and before OCN_R, using:
1809016
kilobytes
starting: ocnfuns.hycom_to_roms_latlon(OCN,RMG)
before interp to roms grid, using:
1809016
kilobytes
doing:
zeta
doing:
temp
doing:
salt
doing:
u
doing:
v
before rotating urm, using:
5497260
kilobytes


: 

In [4]:
print(np.shape(OCN_R['urm']))

(21, 40, 390, 252)


In [ ]:

# %%
# get the OCN_IC dictionary
OCN_IC = ocnfuns.ocn_r_2_ICdict(OCN_R,RMG)
print('driver_run_forecast_LV1: done with ocn_r_2_ICdict')
# add OCN_IC.nc plotting function here !!!!
ic_file_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_ini_file']
ocnfuns.ocn_roms_IC_dict_to_netcdf(OCN_IC, ic_file_out)



In [ ]:
# %%
# get the OCN_BC dictionary
bc_file_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_bc_file']
OCN_BC = ocnfuns.ocn_r_2_BCdict(OCN_R,RMG)
print('driver_run_forecast_LV1: done with ocn_r_2_BCdict')
ocnfuns.ocn_roms_BC_dict_to_netcdf(OCN_BC, bc_file_out)


In [7]:
# make a switch to see if this file exists. If it exists, we don't need to run the code in this block
# first the atm data
# get the data as a dict
# need to specify hhmm because nam forecast are produced at 6 hr increments
ATM = atmfuns.get_atm_data_as_dict(yyyymmdd,hhmm,run_type,atm_mod,'open_dap_nc',PFM)
# put in a function to check to make sure that all the data is good
# put in a function to plot the raw atm data if we want to


# plot some stuff
pltfuns.plot_atm_fields(ATM, RMG, PFM)
print('done with plotting ATM fields')



getting atm forecast for:
[datetime.datetime(2024, 7, 29, 12, 0)
 datetime.datetime(2024, 7, 29, 15, 0)
 datetime.datetime(2024, 7, 29, 18, 0)
 datetime.datetime(2024, 7, 29, 21, 0)
 datetime.datetime(2024, 7, 30, 0, 0) datetime.datetime(2024, 7, 30, 3, 0)
 datetime.datetime(2024, 7, 30, 6, 0) datetime.datetime(2024, 7, 30, 9, 0)
 datetime.datetime(2024, 7, 30, 12, 0)
 datetime.datetime(2024, 7, 30, 15, 0)
 datetime.datetime(2024, 7, 30, 18, 0)
 datetime.datetime(2024, 7, 30, 21, 0)
 datetime.datetime(2024, 7, 31, 0, 0) datetime.datetime(2024, 7, 31, 3, 0)
 datetime.datetime(2024, 7, 31, 6, 0) datetime.datetime(2024, 7, 31, 9, 0)
 datetime.datetime(2024, 7, 31, 12, 0)
 datetime.datetime(2024, 7, 31, 15, 0)
 datetime.datetime(2024, 7, 31, 18, 0)
 datetime.datetime(2024, 7, 31, 21, 0) datetime.datetime(2024, 8, 1, 0, 0)]


In [9]:
# put the atm data on the roms grid, and rotate the velocities
# everything in this dict turn into the atm.nc file

ATM_R  = atmfuns.get_atm_data_on_roms_grid(ATM,RMG)
print('done with: atmfuns.get_atm_data_on_roms_grid(ATM,RMG)')
# all the fields plotted with the data on roms grid


pltfuns.plot_all_fields_in_one(ATM, ATM_R, RMG, PFM)
print('done with: pltfuns.plot_all_fields_in_one(ATM, ATM_R, RMG, PFM)')


done with: atmfuns.get_atm_data_on_roms_grid(ATM,RMG)


In [11]:
# output a netcdf file of ATM_R
# make the atm .nc file here.
# fn_out is the name of the atm.nc file used by roms
fn_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_atm_file'] # LV1 atm forcing filename
print('driver_run_forcast_LV1: saving ATM file to ' + fn_out)
atmfuns.atm_roms_dict_to_netcdf(ATM_R,fn_out)
print('driver_run_forecast_LV1:  done with writing ATM file, Current time ', datetime.now())
# put in a function to plot the atm.nc file if we want to
pltfuns.load_and_plot_atm(RMG, PFM)
print('done with pltfuns.load_and_plot_atm(PFM)')



driver_run_forcast_LV1: saving ATM file to /scratch/PFM_Simulations/LV1_Forecast/Forc/LV1_ATM_FORCING.nc
<xarray.Dataset>
Dimensions:     (tair_time: 21, er: 390, xr: 253, pair_time: 21, qair_time: 21,
                 wind_time: 21, rain_time: 21, srf_time: 21, lrf_time: 21,
                 time: 21)
Coordinates:
    lat         (er, xr) float64 28.52 28.53 28.54 28.55 ... 36.38 36.39 36.39
    lon         (er, xr) float64 -120.3 -120.2 -120.2 ... -118.8 -118.8 -118.8
    ocean_time  (time) float64 9.342e+03 9.342e+03 ... 9.344e+03 9.344e+03
  * tair_time   (tair_time) float64 9.342e+03 9.342e+03 ... 9.344e+03 9.344e+03
  * pair_time   (pair_time) float64 9.342e+03 9.342e+03 ... 9.344e+03 9.344e+03
  * qair_time   (qair_time) float64 9.342e+03 9.342e+03 ... 9.344e+03 9.344e+03
  * wind_time   (wind_time) float64 9.342e+03 9.342e+03 ... 9.344e+03 9.344e+03
  * rain_time   (rain_time) float64 9.342e+03 9.342e+03 ... 9.344e+03 9.344e+03
  * srf_time    (srf_time) float64 9.342e+03 9.342

In [ ]:

print('driver_run_forecast_LV1:  now make .in and .sb files')

pfm_driver_src_dir = os.getcwd()
os.chdir('../sdpm_py_util')
make_LV1_dotin_and_SLURM( PFM )

# run command will be
run_slurm_LV1(PFM)

os.chdir(pfm_driver_src_dir)


# postprocess figure generation
# plot fields from his.nc



In [2]:
# ========================================
# OLD and testing stuff is below this line
# ========================================

# note, this function is hard wired to return 2.5 days of data
# also note that the first time of this data is yyyymmdd 12:00Z
# so we grab nam atm forecast data starting at this hour too.
OCN = ocnfuns.get_ocn_data_as_dict(yyyymmdd,run_type,ocn_mod,'ncks',PFM)
print('driver_run_forecast_LV1: done with get_ocn_data_as_dict: Current time ',datetime.now() )
# add OCN plotting function here !!!!



Time to get full file using ncks = 1517.43 sec
Return code = 0 (0=success, 1=skipped ncks)
driver_run_forecast_LV1: done with get_ocn_data_as_dict: Current time  2024-07-31 13:19:42.922055


In [6]:
import xarray as xr

fn='/scratch/PFM_Simulations/LV1_Forecast/Forc/hy_cat_2024-07-29T12:00.nc'
ds = xr.open_dataset(fn)
print(ds.lat.values)


[28.         28.04000092 28.07999992 28.12000084 28.15999985 28.20000076
 28.23999977 28.28000069 28.31999969 28.36000061 28.39999962 28.44000053
 28.47999954 28.52000046 28.55999947 28.60000038 28.63999939 28.68000031
 28.71999931 28.76000023 28.79999924 28.84000015 28.87999916 28.92000008
 28.95999908 29.         29.04000092 29.07999992 29.12000084 29.15999985
 29.20000076 29.23999977 29.28000069 29.31999969 29.36000061 29.39999962
 29.44000053 29.47999954 29.52000046 29.55999947 29.60000038 29.63999939
 29.68000031 29.71999931 29.76000023 29.79999924 29.84000015 29.87999916
 29.92000008 29.95999908 30.         30.04000092 30.07999992 30.12000084
 30.15999985 30.20000076 30.23999977 30.28000069 30.31999969 30.36000061
 30.39999962 30.44000053 30.47999954 30.52000046 30.55999947 30.60000038
 30.63999939 30.68000031 30.71999931 30.76000023 30.79999924 30.84000015
 30.87999916 30.92000008 30.95999908 31.         31.04000092 31.07999992
 31.12000084 31.15999985 31.20000076 31.23999977 31

In [9]:
import os
import glob
tstr = "2024-07-30T12:00"
ffname = "hy_" + tstr + "_*.nc"
full_fns_out = PFM['lv1_forc_dir'] + "/" + ffname
ncfiles = glob.glob(full_fns_out)
cat_fname = PFM['lv1_forc_dir'] + '/' + 'hy_cat_' + tstr + '.nc'

print(ncfiles)


[]


In [29]:
from datetime import timedelta

tstr = '2024-07-30T12:00'
print(tstr)
t1 = datetime.strptime(tstr,"%Y-%m-%dT%H:%M")
t2 = t1 + 2.5 * timedelta(days=1)
print(t1)
print(t2)
t2str = t2.strftime("%Y-%m-%dT%H:%M")
print(t2str)

2024-07-30T12:00
2024-07-30 12:00:00
2024-08-02 00:00:00
2024-08-02T00:00


In [ ]:
# a block of code to use ncks...
import time
import subprocess

vstr = 'surf_el,water_temp,salinity,water_u,water_v,depth'

# make the ocn IC and BC .nc files here
# fn*_out are the names of the the IC.nc and BC.nc roms files
lv1_forc_dir = PFM['lv1_forc_dir']   #'/Users/mspydell/research/FF2024/models/SDPM_mss/atm_stuff/ocn_test_IC_file.nc'
full_fn_out = lv1_forc_dir + '/hycom_test_mss.nc'

west =  PFM['latlonbox'][2]+360.0
east =  PFM['latlonbox'][3]+360.0
south = PFM['latlonbox'][0]
north = PFM['latlonbox'][1]

yyyy = yyyymmdd[0:4]
mm = yyyymmdd[4:6]
dd = yyyymmdd[6:8]

# time limits
dstr0 = yyyy + '-' + mm + '-' + dd + 'T12:00'
dstr1 = yyyy + '-' + mm + '-' + str( int(dd) + 1 ) + 'T00:00'

url='https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0/FMRC/runs/' 
url2 = 'GLBy0.08_930_FMRC_RUN_' + yyyy + '-' + mm + '-' + dd + 'T12:00:00Z' 
url3 = url + url2
cmd_list = ['ncks',
    '-d', 'time,'+dstr0+','+dstr1,
    '-d', 'lon,'+str(west)+','+str(east),
    '-d', 'lat,'+str(south)+','+str(north),
    '-v', vstr,
    url3 ,
    '-4', '-O', full_fn_out]

print(cmd_list)

# run ncks
tt0 = time.time()
ret1 = subprocess.call(cmd_list)


In [51]:
import xarray as xr
from datetime import timedelta

ds = xr.open_dataset(full_fn_out)
#print(ds)

t_hy = ds.time
t_ref = PFM['modtime0']

dt_day = (t_hy - np.datetime64(t_ref))  / np.timedelta64(1,'D')

print(dt_day.values)
#print(t_hy[0]-t_ref)


#t_hy2 = datetime(t_hy)

#t_ref?
#deltat = t_hy - t_ref

#print(deltat)


#print(t_hy)
#print(t_ref)
      


[9340.5   9340.625 9340.75  9340.875 9341.   ]


In [5]:
zz=np.random.randn(2,3,4,5)
dum = zz[1,2,:,:]
ang = np.random.randn(4,5)
ang2 = np.tile(ang,(2,3,1,1))


[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
